# AWS DevDay Seoul 2019
## 모두를 위한 컴퓨터 비전 딥러닝 툴킷, GluonCV 따라하기
### Lab 3. Object detection 모델을 AWS DeepLens에 배포하기

#### 랩 순서

1. GluonCV 모델을 DeepLens에 배포할 모델 형태로 준비하기
2. DeepLens 모델 정의하기
3. 제공된 DeepLens 프로젝트에서 모델 교체하기
4. DeepLens에 배포하고 테스트하기

DeepLens에 배포한 모델을 저장할 S3 버킷이름과 모델 파일 이름을 정합니다.

> 주의: S3 버킷 이름은 꼭 **"deeplens-"**로 시작해야합니다.

In [2]:
S3_BUCKET = 'deeplens-mh-s3-us-east-1'
DL_MODEL_NAME = 'model-yolov3.tar.gz'

In [3]:
!aws s3 mb s3://$S3_BUCKET --region us-east-1

make_bucket: deeplens-mh-s3-us-east-1


### Task 1 - GluonCV 모델을 DeepLens에 배포할 모델 형태로 준비하기

In [4]:
import mxnet as mx
import gluoncv as gcv

In [5]:
net_name = 'yolo3_darknet53_custom'
model_param_fname = '../lab2-gluoncv/yolo3_darknet53_voc_0000.params'
classes = ['bird', 'boat', 'car', 'person']

ctx = mx.gpu()

In [6]:
net = gcv.model_zoo.get_model(net_name, classes=classes, ctx=ctx)
net.load_parameters(model_param_fname, ctx=ctx)

net.hybridize()

x=mx.ndarray.zeros([1, 3, 512, 512], ctx=ctx)
y=net(x)

mx.nd.waitall()
net.export('deeplens-yolov3')

모델의 심볼 파일(.json)과 파라메터 파일(.params)이 잘 생성되었는지 확인하고, tar.gz 형태로 압축한 후 위에서 지정한 S3 bucket으로 업로드합니다.

> **주의** 강연자의 별도의 안내가 있을 경우, 지급된 USB 메모리스틱에 압축 파일을 복사하고, S3 버킷 업로드는 생략합니다.

In [7]:
!ls -l deeplens-yolov3*

-rw-rw-r-- 1 ec2-user ec2-user 246794208 Sep 24 04:18 deeplens-yolov3-0000.params
-rw-rw-r-- 1 ec2-user ec2-user    217006 Sep 24 04:18 deeplens-yolov3-symbol.json


In [8]:
!tar cvfz ./$DL_MODEL_NAME ./deeplens-yolov3-*
!aws s3 cp ./$DL_MODEL_NAME s3://$S3_BUCKET/models/

./deeplens-yolov3-0000.params
./deeplens-yolov3-symbol.json
upload: ./model-yolov3.tar.gz to s3://deeplens-mh-s3-us-east-1/models/model-yolov3.tar.gz


### Task 2 - DeepLens 모델 정의하기

> **주의** 강연자의 별도의 안내가 있을 경우, Task 2를 수행하지 않고 Task 3을 수행합니다.

1. DeepLens 콜솔로 이동 후, 'Resources > Models' 선택
2. 'Import model' 버튼 클릭

![](./images/devday-deeplens-lab-01.png)

3. 'Import source'를 'Externally trained model'로 변경
4. Model settings에 값들을 입력
- Model artifact path: s3://__S3_BUCKET__/models/model-yolov3.tar.gz
- Model name: model 이름 (예, object-detection)
- Model framework: MXNet 선택

![](./images/devday-deeplens-lab-02.png)

### Task 3 - 제공된 DeepLens 프로젝트에서 모델 교체하기

1. DeepLens 콜솔로 이동 후, 'Resources > Projects' 선택하고,
2. 프로젝트 목록에서 'devday' 클릭합니다.

![](./images/devday-deeplens-lab-06.png)

3. 모델 변경을 위해서 Model의 'Remove' 버튼 클릭

![](./images/devday-deeplens-lab-08.png)

4. GluonCV로 학습시킨 모델을 추가하기 위해서, 'Add model' 클릭

![](./images/devday-deeplens-lab-09.png)

5. 모델 목록에서, 이전 단계에서 정의한 모델 선택 후 'Add model' 버튼 클ㄹ릭

![](./images/devday-deeplens-lab-10.png)

DeepLens에 연결된 모니터을 통해서 결과를 확인하기 위해서는 DeepLens에 로그인해서 아래 명령을 수행합니다.

```
mplayer -demuxer lavf /opt/awscam/out/ch1_out.h264
```

### Task 3 - 모델 파일을 직접 DeepLens에 옮기기

> 이 단계는 DeepLens 콘솔을 통한 모델 배포가 아닌, DeepLens에 직접 모델을 복사하여 실행하는 방법을 설명합니다.

1. DeepLens에 직접 로그인

2. USB에 있는 모델 파일 복사


```
$ sudo bash
<비밀번호 입력>

# cd /media/aws_cam/*
# ls
<모델 파일 이름 확인>

# cp <모델 파일 이름> /opt/awscam/greengrass/ggc/deployment/mlmodel/<디렉토리 이름 업데이트 예정>

```

3. Greengrass 서비스 재시작

```
sudo systemctl restart greengrassd.service
```

4. 수행 결과 확인

DeepLens에 연결된 모니터을 통해서 결과를 확인하기 위해서는 DeepLens에 로그인해서 아래 명령을 수행합니다.

```
mplayer -demuxer lavf /opt/awscam/out/ch1_out.h264
```